In [107]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.preprocessing import Normalizer, normalize, RobustScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from functools import partial
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import make_scorer,r2_score, mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr, spearmanr
from pathlib import Path
from joblib import load, dump
import optuna

from rdkit.Chem import MolFromSmiles, MolToSmiles
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect

from descriptastorus.descriptors import rdNormalizedDescriptors, AtomPairCounts

from scipy import stats
from scipy.stats import norm
from statsmodels.graphics.gofplots import qqplot

In [2]:
np.random.seed(5)

In [3]:
sns.set(rc={'figure.figsize': (12, 10)})
sns.set_style('whitegrid')
sns.set_context('paper',font_scale=1.5)

# Load Data

In [4]:
SAVE_DIR = Path('../models/LOGS/random_split/PropertyFP_reduced/aqsoldb')
SAVE_DIR.mkdir(exist_ok=True,parents=True)

In [5]:
data = pd.read_csv('../data/AqSOLDB/curated-solubility-dataset_processed.csv')
data.head()

,ID,Name,InChI,InChIKey,SMILES,Solubility,SD,Ocurrences,Group,MolWt,...,NumValenceElectrons,NumAromaticRings,NumSaturatedRings,NumAliphaticRings,RingCount,TPSA,LabuteASA,BalabanJ,BertzCT,processed_smiles
0,A-3,"N,N,N-trimethyloctadecan-1-aminium bromide",InChI=1S/C21H46N.BrH/c1-5-6-7-8-9-10-11-12-13-...,SZEMGTQCPRNXEG-UHFFFAOYSA-M,[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127,0.0,1,G1,392.510,...,142.0,0.0,0.0,0.0,0.0,0.00,158.520601,0.000000,210.377334,CCCCCCCCCCCCCCCCCC[N+](C)(C)C
1,A-4,Benzo[cd]indol-2(1H)-one,InChI=1S/C11H7NO/c13-11-8-5-1-3-7-4-2-6-9(12-1...,GPYLCFQEKPUWLD-UHFFFAOYSA-N,O=C1Nc2cccc3cccc1c23,-3.254767,0.0,1,G1,169.183,...,62.0,2.0,0.0,1.0,3.0,29.10,75.183563,2.582996,511.229248,O=C1Nc2cccc3cccc1c23
2,A-5,4-chlorobenzaldehyde,InChI=1S/C7H5ClO/c8-7-3-1-6(5-9)2-4-7/h1-5H,AVPYQKSLYISFPO-UHFFFAOYSA-N,Clc1ccc(C=O)cc1,-2.177078,0.0,1,G1,140.569,...,46.0,1.0,0.0,0.0,1.0,17.07,58.261134,3.009782,202.661065,O=Cc1ccc(Cl)cc1
3,A-9,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,InChI=1S/C25H30N2O4/c1-5-20(26(10-22-14-28-22)...,FAUAZXVRLVIARB-UHFFFAOYSA-N,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065,0.0,1,G1,422.525,...,164.0,2.0,4.0,4.0,6.0,56.60,183.183268,1.084427,769.899934,c1cc(N(CC2CO2)CC2CO2)ccc1Cc1ccc(N(CC2CO2)CC2CO...
4,A-10,vinyltoluene,"InChI=1S/C9H10/c1-3-9-6-4-5-8(2)7-9/h3-7H,1H2,2H3",JZHGRUMIRATHIU-UHFFFAOYSA-N,Cc1cccc(C=C)c1,-3.123150,0.0,1,G1,118.179,...,46.0,1.0,0.0,0.0,1.0,0.00,55.836626,3.070761,211.033225,C=Cc1cccc(C)c1


# Featurize

In [ ]:
def getfp(mol, nBits=1024,radius=2):
    mol = MolFromSmiles(mol)

    fp = GetMorganFingerprintAsBitVect(mol,radius=radius,nBits=nBits)
    return np.array(fp).reshape(-1,nBits)

In [ ]:
generator = rdNormalizedDescriptors.RDKit2DNormalized()
generator_atompair = AtomPairCounts()

In [ ]:
features = [x[0] for x in generator.columns]
len(features)

In [ ]:
features = [x[0] for x in generator.columns]
len(features)
features_df = pd.DataFrame(np.concatenate(generator.processSmiles(data.processed_smiles.tolist())[1]).reshape(data.shape[0],-1)[:, 1:],columns=features)

In [ ]:
features_df = pd.DataFrame(features_df,columns=features)

In [ ]:
features_df.columns[features_df.isnull().any().values]

We do have missing values. Let's see who is causing this...

In [ ]:
features_df.loc[features_df['MaxAbsPartialCharge'].isnull(),'value_is_NaN'] = 'Yes'

In [ ]:
features_df[~features_df['value_is_NaN'].isnull()]

In [ ]:
wrong_idx = features_df[~features_df['value_is_NaN'].isnull()].index

Just one molecule for which charges could not be calculated. Let's take a look into its structure and SMILES

In [ ]:
data.iloc[wrong_idx]

In [ ]:
MolFromSmiles('[C-]#[O+]')

From the MolWT x LogS plot, it doesnt seem to be an outlier. I think we just remove it, just in case...

In [ ]:
data.drop(wrong_idx,inplace=True)
features_df.drop(wrong_idx,inplace=True)

In [ ]:
slic = data[['ID', 'Name', 'InChI', 'InChIKey', 'SMILES', 'processed_smiles', 'Solubility', 'SD',
       'Ocurrences', 'Group']]

In [ ]:
data_feat = pd.concat([slic, features_df],axis=1)

In [ ]:
data_feat.drop('value_is_NaN',axis=1,inplace=True)

In [ ]:
data_feat.to_csv('../data/AqSOLDB/curated-solubility-dataset_processed_rdki2dnormalized.csv',index=False)

In [ ]:
data_feat.columns[data_feat.isnull().any().values]

In [ ]:
data_feat.head()

Let's remove the outlier A-2512. 

In [ ]:
data_feat = data_feat[data_feat['ID']!="A-2512"]

In [ ]:
plt.figure(figsize=(12,8))
ax=sns.scatterplot(*embedding.T,s=90, hue=hu, palette='seismic')


norm = plt.Normalize(data_feat['Solubility'].min(), data_feat['Solubility'].max())
sm = plt.cm.ScalarMappable(cmap="seismic", norm=norm)
sm.set_array([])
plt.legend('LogS')
# Remove the legend and add a colorbar
ax.get_legend().remove()
ax.set_xlabel('UMAP1')
ax.set_ylabel('UMAP2')
ax.figure.colorbar(sm)

# Feature selection

Since we wil be using A LOT of physicochemical descriptors (e.g. molecular weight, TPSA, chi values etc), let's check which ones we could ignore before training different models. This might help us train faster and include only powerful predictions in our data. In addition, it might help us avoid overfitting.

In [ ]:
esol_x = data_feat[features].values
esol_y = data_feat['Solubility'].values

print(esol_x.shape, esol_y.shape)

### Mutal information

[Mutual information](https://thuijskens.github.io/2017/10/07/feature-selection/) is a measure between two (possibly multi-dimensional) random variables $X$ and $Y$, that quantifies the amount of information obtained about one random variable, through the other random variable. The mutual information is given by:

$$
\begin{align}
I(X; Y) = \int_X \int_Y p(x, y) \log \frac{p(x, y)}{p(x) p(y)} dx dy
\end{align}
$$

If the joint distribution $p(x,y)$ of variables $X$ and $Y$ equals the individual probabilities $p(x)$ and $p(y)$, the variables are considered independent the integral is 0. Therefore, our goal is to find variables that are somehow correlated with the dependent variable logS. 

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression

In [ ]:
importances = mutual_info_regression(esol_x, esol_y)

In [ ]:
df_importances = pd.DataFrame(importances,index=features)

In [ ]:
features[0:5]

In [ ]:
df_importances.sort_values(0,ascending=False).head(20)

In [ ]:
most_relevant_mi = df_importances.sort_values(0,ascending=False).head(20).index.tolist()
most_relevant_mi[0:5]

At first, it seems that the top-20 most correlated features with logS makes sense. For instance, the LogP is definitely important to define solubility, with more lipophilic molecules being less water-soluble. 
After logP, 0 and 1-order chi descriptors seems to be the most correlated. This chi descriptors essentialy encode the topology of a molecule, including its valence and sigma electrons, lone pairs the the degree of branching. In a similar way, the molecular refractivity (MolMR) encodes the steric bulk of a molecule because it is calculated from the molecular volume. 
In summary, the top 20 most correlated descriptors are properties that impact the solubility of molecules in some way. 

In [ ]:
df_importances.sort_values(0,ascending=False).tail(20)

For the bottom 20 features we can see that the are mostly related to fractions of specific chemical groups, which does tells us something about the structure but are not determinant for solubility per se. For instance, a molecule with nitro groups might not be soluble even considering the high polarility of this group. 

Ok, that already give us some ideas about which descriptors should be in the logS estimator. In a future next step, we'll further filter this collection of features. For now,  let's select any feature with MI > 0.0.

In [ ]:
selected_features = df_importances[df_importances[0]>0].index.tolist()#np.array(features)[selectkbest.get_support()]

In [ ]:
selected_features

In [ ]:
data_feat[selected_features]

**Now for something different: Random Forest**

We could try all feature selection methods from scikit-learn but one method that I prefer to use is random forest (RF). Random forest is a learning algorithm that uses the average response of an ensemble of decision trees to make a prediction. Not only data, but the algorithm reduces overfitting but using different data samples and features at each tree, which essentialy decorrelates each predictor. Since the underlying model uses splits based on different input features, we can ask the model which features it considered the most important to make a split with the ```feature_importance_``` variable.

In order to validate the predictions, we'll use out-of-the-bag samples as a test set.

In [ ]:
feat_model = RandomForestRegressor(2000,n_jobs=-1, oob_score=True)

In [ ]:
feat_model.fit(data_feat[selected_features].values, esol_y)

In [ ]:
oob_preds = feat_model.oob_prediction_

In [ ]:
oob_preds.shape, esol_y.shape

In [ ]:
def rf_feat_importance(model, feats_names):
    return pd.DataFrame({'cols':feats_names, 'imp':model.feature_importances_}).sort_values('imp',ascending=False)

In [ ]:
len(selected_features)

In [ ]:
fi = rf_feat_importance(feat_model, selected_features)

In [ ]:
df_importances.sort_values(0,ascending=False)

In [ ]:
fi.head(20)

In [ ]:
def plot_fi(fi):
    return fi.plot('cols','imp','barh',figsize=(16,12),legend=False)

In [ ]:
plot_fi(fi[:25])

Again, the most relevant descriptors make sense. For instance, the top predictor of solubility is the logP (lipophilicity of the molecule). Other importante descriptors include molar refractivity, partial charge, some topological descriptors (e.g. Chi, BertzCT, SMR_VSA10), the total polar surface area and the number of heteroatoms. However, logP **DOMINATES** all other features by a large margin. This is consistent with the original [ESOL publication](https://doi.org/10.1021/ci034243x).

## Selecting features

Let's try selecting only features if their importance is **> 5% (0.005)**

In [ ]:
most_relevant_features = fi[fi['imp']>=0.005]
most_relevant_features_idx = most_relevant_features.index.tolist()

In [ ]:
most_relevant_features

Recently I learned in the FASTAI course how to cluster the features in order to further the features. If two feature are highly correlated, we can eliminate one of them or maybe merge them into a unique feature. Let's try plotting a dendrogram:

In [ ]:
import scipy
from scipy.cluster import hierarchy as hc

def cluster_columns(df, figsize=(12,16), font_size=18):
    corr = np.round(scipy.stats.spearmanr(df).correlation, 4)
    corr_condensed = hc.distance.squareform(1-corr)
    z = hc.linkage(corr_condensed, method='average')
    fig = plt.figure(figsize=figsize)
    hc.dendrogram(z, labels=df.columns, orientation='left', leaf_font_size=font_size)
    plt.tight_layout()
    plt.xticks(fontsize=18)

In [ ]:
xs_imps = pd.DataFrame(data[most_relevant_features.cols.values.tolist()],columns=most_relevant_features.cols.values.tolist()).reset_index(drop=True)

In [ ]:
xs_imps

In [ ]:
cluster_columns(xs_imps)
#plt.savefig(SAVE_DIR/'feature_importance_clustering.png',dpi=600)

In [ ]:
plt.figure(figsize=(24,20))
sns.set_context(font_scale=1.5)
ax=sns.heatmap(xs_imps.corr(), cmap='magma', annot=True)

The Chi, BertzCT and MolMR are the most correlated descriptors. In addition, MinPartialCharge and MaxAbsPartialCharge charge are the most correlated features. 
Surprisingly, the [quantitative estimation of drug-likeness - qed](https://www.rdkit.org/docs/source/rdkit.Chem.QED.html) shows up among the most relevant features! The qed value basically incorporates many physicochemical properties, such as logP, molecular weight, TPSA, HBD and HBA in order to estimate the druglikeness of a molecule. But wait a second... We already have some of these properties among out selected features! No wonder qed is in the same cluster as MolLogP, chi descriptors, max partial charges and molar refractivity. It actually embed these features into a single value. 

## Pruning the tree a little bit more...

This time we'll use recursive feature elimination in order to select the best set of features based on RF feature importance.

In [ ]:
from sklearn.feature_selection import RFECV

In [ ]:
pruner = RandomForestRegressor(2000,n_jobs=-1)
selector = RFECV(pruner, step=1, cv=5, min_features_to_select=5)
selector.fit(data[most_relevant_features.cols],esol_y)

In [ ]:
selector.grid_scores_.mean()

In [ ]:
selector.n_features_

In [ ]:
pruned_list = most_relevant_features.cols.values[selector.get_support()]

In [ ]:
most_relevant_features.cols.values

In [ ]:
cluster_columns(data[pruned_list])
#plt.savefig(SAVE_DIR/'feature_importance_clustering.png',dpi=600)

In [ ]:
pruned_list

The Chi4v and FpDensityMorgan1 features were removed. Well, there are many lower-tier chi descriptors already present and the contribution of density of circular fingerprints doesnt really give any new information compared to topological descriptors, including chi. By the dendrogram, it seems we could also remove Chi1V or MolMR. 

In [ ]:
def get_oob(df):
    m = RandomForestRegressor(2000, min_samples_leaf=15, max_samples=100, max_features=0.5,n_jobs=-1, oob_score=True)
    m.fit(df,esol_y)
    return m.oob_score_

In [ ]:
get_oob(data[pruned_list].values), 

In [ ]:
{c: get_oob(data[pruned_list].drop(c,axis=1)) for c in data[pruned_list].columns}

Now let's try removing Chi1v and MaxAbsPartialCharge

In [ ]:
#plot_fi(rf_feat_importance(m,pruned_list))
to_drop = ['Chi1v','MaxAbsPartialCharge', 'Chi4n','Chi4v','Chi3v','BertzCT','FpDensityMorgan1']

In [ ]:
to_keep

In [ ]:
plt.figure(figsize=(24,20))
sns.set_context(font_scale=1.5)
ax=sns.heatmap(xs_imps.drop(to_drop,axis=1).corr(), cmap='magma', annot=True)
plt.savefig('../models/LOGS/features_heatmap.png',dpi=600)

In [ ]:
cluster_columns(xs_imps[to_keep])
plt.savefig('../models/LOGS/features_dendrogram.png',dpi=600)

In [ ]:
to_keep = list(xs_imps.drop(to_drop,axis=1).columns)

In [ ]:
to_keep

In [ ]:
get_oob(data[to_keep]) 

## Final dataset

Ok, so our final dataset consists of 9 features. It seems it removed all highly redudant features and only selected the most informative ones. 

In [ ]:
pruned_data = data[data.columns[0:11].tolist() + to_keep.tolist()]
pruned_data.head()

In [ ]:
pruned_data.drop('Unnamed: 0',axis=1,inplace=True)

# References

**Detecting outliers**

https://www.ucd.ie/ecomodel/Resources/QQplots_WebVersion.html

https://www.dummies.com/programming/big-data/data-science/graphical-tests-of-data-outliers/

https://machinelearningmastery.com/how-to-use-statistics-to-identify-outliers-in-data/

https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba

https://www.dbs.ifi.lmu.de/Publikationen/Papers/LOF.pdf

https://towardsdatascience.com/local-outlier-factor-for-anomaly-detection-cc0c770d2ebe


**Normality tests**

Machine learning mastery post: https://machinelearningmastery.com/a-gentle-introduction-to-normality-tests-in-python/

Shapiro-Wilk test: https://www.itl.nist.gov/div898/handbook/prc/section2/prc213.htm

Anderson-Darling test : https://www.itl.nist.gov/div898/handbook/prc/section2/prc21.htm


GraphPad entry: https://www.graphpad.com/guides/prism/latest/statistics/stat_choosing_a_normality_test.htm